In [1]:
!conda install -y bedtools bioawk samtools

Retrieving notices: ...working... done
Channels:
 - conda-forge
 - bioconda
 - defaults
Platform: linux-64
Solving environment: done

## Package Plan ##

  environment location: /root/conda

  added / updated specs:
    - bedtools
    - bioawk
    - samtools


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    bedtools-2.31.1            |       hf5e1c6e_1         1.5 MB  bioconda
    bioawk-1.0                 |      he4a0461_10         198 KB  bioconda
    htslib-1.20                |       h81da01d_0         2.9 MB  bioconda
    libcurl-8.7.1              |       h251f7ec_0         424 KB
    libdeflate-1.20            |       hd590300_0          70 KB  conda-forge
    libstdcxx-ng-13.2.0        |       hc0a3c3a_7         3.7 MB  conda-forge
    ncurses-6.5                |       h59595ed_0         867 KB  conda-forge
    samtools-1.20              |       h50ea8bc_0         459 KB  bioco

In [2]:
%cd /mnt/c/Users/user/Desktop/과제/binfo1-work

/mnt/c/Users/user/Desktop/과제/binfo1-work


/root/.local/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [3]:
!grep '	start_codon	.*	+	.*transcript_support_level "1"' ../binfo1-datapack1/gencode.gtf | \
 sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-start.gtf
!head gencode-start.gtf

grep: ../binfo1-datapack1/gencode.gtf: No such file or directory


In [4]:
!grep '	exon	.*	+	' ../binfo1-datapack1/gencode.gtf | \
 sed -e 's/	[^	]*transcript_id "\([^"]*\)".*$/	\1/g' > gencode-plusexon.gtf
!head gencode-plusexon.gtf

grep: ../binfo1-datapack1/gencode.gtf: No such file or directory


In [5]:
!bedtools intersect -a gencode-start.gtf -b gencode-plusexon.gtf -wa -wb | \
 awk -F'	' -v OFS='	' '$9 == $18 { print $10, $13-1, $14, $18, $4-1, $16; }' | \
 sort -k1,1 -k2,3n -k4,4 > gencode-exons-containing-startcodon.bed
!head gencode-exons-containing-startcodon.bed; tail gencode-exons-containing-startcodon.bed

In [6]:
!(samtools view -H ../binfo1-datapack1/RPF-siLuc.bam; \
  samtools view -F20 ../binfo1-datapack1/RPF-siLuc.bam | \
  bioawk -c sam '{ if (length($seq) >= 25) print $0; }') | \
 samtools view -b -o filtered-RPF-siLuc.bam

In [7]:
!ls -al *RPF-siLuc.bam

-rwxrwxrwx 1 root root 1055123479 May  6  2021 RPF-siLuc.bam
-rwxrwxrwx 1 root root  554373659 May 10 10:46 filtered-RPF-siLuc.bam


In [8]:
!bedtools genomecov -ibam filtered-RPF-siLuc.bam -bg -5 > fivepcounts-RPF-siLuc.bed
!head fivepcounts-RPF-siLuc.bed

chr1	3176534	3176535	1
chr1	3196256	3196257	1
chr1	3221526	3221527	1
chr1	3221542	3221543	2
chr1	3221545	3221546	3
chr1	3221546	3221547	2
chr1	3221548	3221550	1
chr1	3221571	3221572	1
chr1	3221897	3221898	1
chr1	3221994	3221995	1


In [9]:
!bedtools intersect -a fivepcounts-RPF-siLuc.bed -b gencode-exons-containing-startcodon.bed \
 -wa -wb -nonamecheck > fivepcounts-filtered-RPF-siLuc.txt
!head fivepcounts-filtered-RPF-siLuc.txt